# 加权注意力

In [29]:
import re
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## Foundation

### Tensor

In [30]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = 0
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return str(self.data)

    def __add__(self, other):
        p = Tensor(self.data + other.data)

        def gradient_fn():
            self.grad += p.grad
            other.grad += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __sub__(self, other):
        p = Tensor(self.data - other.data)

        def gradient_fn():
            self.grad += p.grad
            other.grad += -p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __mul__(self, other):
        p = Tensor(self.data * other.data)

        def gradient_fn():
            self.grad += p.grad * other.data
            other.grad += p.grad * self.data

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __truediv__(self, other):
        p = Tensor(self.data / other.data)

        def gradient_fn():
            self.grad += p.grad / other.data
            other.grad += -p.grad * self.data / (other.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __matmul__(self, other):
        p = Tensor(np.matmul(self.data, other.data))

        def gradient_fn():
            self.grad += np.matmul(p.grad, other.data.swapaxes(-1, -2))
            other.grad += np.matmul(self.data.swapaxes(-1, -2), p.grad)

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def transpose(self, axes=None):
        p = Tensor(np.transpose(self.data, axes))

        def gradient_fn():
            if axes is None:
                self.grad += np.transpose(p.grad)
            else:
                idx = np.argsort(axes)
                self.grad += np.transpose(p.grad, idx)

        p.gradient_fn = gradient_fn
        p.parents = {self}
        return p

    @property
    def T(self):
        return self.transpose()

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)
            self.grad += grad[0]
            other.grad += grad[1]

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

### Base Dataset

In [31]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size
        return Tensor(self.features[start: end]), Tensor(self.labels[start: end])

    @abstractmethod
    def estimate(self, predictions):
        pass

### Base Layer

In [32]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, *args):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

## Data

### LLM Dataset

In [33]:
class LLMDataset(Dataset):

    def __init__(self, filename, batch_size, stride=1):
        self.filename = filename
        self.stride = stride
        super().__init__(batch_size)

    def load(self):
        with open(self.filename, 'r', encoding='utf-8') as f:
            self.text = f.read().lower()

        self.vocabulary = sorted(set(self.split_text(self.text)))
        self.vocabulary.extend(['<|eos|>', '<|unk|>'])
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = self.encode(self.text)
        self.batchs = []
        for i in range(1, len(self.tokens) - self.batch_size, self.stride):
            self.batchs.append(self.tokens[i: i + self.batch_size])

    @staticmethod
    def split_text(text):
        words = re.split(r'([,.:;?_!"()\']|\s)', text.lower())
        return [t.strip() for t in words if t.strip()]

    def train(self):
        self.features = []
        self.labels = []
        for i in range(1, len(self.batchs) * 8 // 10):
            self.features.append(self.batchs[i])
            self.labels.append(self.onehot(self.batchs[i + 1]))

    def eval(self):
        self.features = []
        self.labels = []
        for i in range(len(self.batchs) * 8 // 10 + 1, len(self.batchs) - 1):
            self.features.append(self.batchs[i])
            self.labels.append(self.onehot(self.batchs[i + 1]))

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return Tensor(self.features[index]), Tensor(self.labels[index])

    def encode(self, text):
        words = self.split_text(text)
        words = [word if word in self.word2index else '<|unk|>' for word in words]
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        text = " ".join([self.index2word[index] for index in tokens])
        text = re.sub(r'\s+([,.:;?!)\]}>])', r'\1', text)
        text = re.sub(r'([([<{])\s+', r'\1', text)
        text = re.sub(r'(")\s+(.*?)\s+(")', r'\1\2\3', text)
        text = re.sub(r"(')\s+(.*?)\s+(')", r'\1\2\3', text)
        return text.strip()

    def onehot(self, tokens):
        ebd = np.zeros((len(tokens), len(self.vocabulary)))
        ebd[np.arange(len(tokens)), tokens] = 1
        return ebd

    def estimate(self, predictions):
        pass

## Model

### Linear Layer

In [34]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.randn(out_size, in_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data / len(x.data)
            self.bias.grad += np.sum(p.grad, axis=0) / len(x.data)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### Embedding Layer

In [35]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=None):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.rand(embedding_size, vocabulary_size) / vocabulary_size)

    def forward(self, x: Tensor):
        weights = self.weight.data.T[x.data]
        p = Tensor(np.sum(weights, axis=self.axis) if self.axis is not None else weights)

        def gradient_fn():
            if type(self.weight.grad) is not np.ndarray:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

### Softmax Activation Function

In [36]:
class Softmax(Layer):

    def __init__(self, axis=-1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            grad = np.sum(p.data * p.grad, axis=self.axis, keepdims=True)
            x.grad += p.data * (p.grad - grad)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### GPT Embedding Layer

In [37]:
class GPTEmbedding(Layer):

    def __init__(self, vocabulary_size, context_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.context_size = context_size
        self.embedding_size = embedding_size

        self.token_embedding = Embedding(self.vocabulary_size, self.embedding_size)
        self.positional_embedding = Embedding(self.context_size, self.embedding_size)

    def forward(self, x: Tensor):
        token = self.token_embedding(x)
        position = self.positional_embedding(Tensor(range(len(x.data))))
        return token + position

### GPT Attention Layer

In [38]:
class GPTAttention(Layer):

    def __init__(self, embedding_size):
        super().__init__()
        self.embedding_size = embedding_size

        self.attention_query = Linear(self.embedding_size, self.embedding_size)
        self.attention_key = Linear(self.embedding_size, self.embedding_size)
        self.attention_value = Linear(self.embedding_size, self.embedding_size)
        self.softmax = Softmax()

    def forward(self, x: Tensor):
        query = self.attention_query(x)
        key = self.attention_key(x)
        value = self.attention_value(x)

        scores = query @ key.T
        weights = self.softmax(scores)
        return weights @ value

### GPT Layer

In [39]:
class GPT(Layer):

    def __init__(self, vocabulary_size, context_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.context_size = context_size
        self.embedding_size = embedding_size

        self.embedding = GPTEmbedding(self.vocabulary_size, self.context_size, self.embedding_size)
        self.attention = GPTAttention(self.embedding_size)

    def forward(self, x: Tensor):
        x = self.embedding(x)
        return self.attention(x)

## Configuration

### Context Size

In [40]:
CONTEXT_SIZE = 8

### Embedding Size

In [41]:
EMBEDDING_SIZE = 3

## Testing

### Estimating

In [42]:
dataset = LLMDataset('../one-day.txt', CONTEXT_SIZE)

model = GPT(len(dataset.vocabulary), CONTEXT_SIZE, EMBEDDING_SIZE)

features, labels = dataset[0]
prediction = model(features)

print('Token shape: ', prediction.shape())
print('Tokens: ', prediction.data)

Token shape:  (8, 3)
Tokens:  [[-0.05991031  0.10588113 -0.01873922]
 [-0.05950963  0.10578051 -0.01910736]
 [-0.05972036  0.10586068 -0.01894778]
 [-0.05968404  0.1058229  -0.0189463 ]
 [-0.05954513  0.10580849 -0.01909918]
 [-0.05975183  0.10583453 -0.01888251]
 [-0.05945361  0.10579151 -0.01919227]
 [-0.05974991  0.10583429 -0.0188843 ]]
